# Passaggi preliminari

In [1]:
# Importazione librerie
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Definizione funzioni

In [2]:
# Funzione scorrimento pagina per caricare tutti i prodotti
def scroll_down(driver):
    """Scroll down to bottom of the page"""
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(3)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [3]:
# Funzione ricerca prodotti e prezzi
def find_all(soup):
    # Ricerca degli elementi desiderati (prezzi e nomi prodotti)
    desc  = soup.find_all('div', class_='description')
    prod  = soup.find_all('span', class_ = 'product-brand')
    # Creazione liste vuote
    prices   = []
    products = []
    # Ricerca e salvataggio nomi prodotti
    for i in prod:
        text = i.get_text(strip=True)
        products.append(text)
    # Ricerca e salvataggio prezzi   
    for element in desc:
        price = element.find('p').text.strip()
        prices.append(price)        
    output = pd.DataFrame({'Product': products,
                           'Price': prices})
    return output

In [4]:
def access(driver):
    # Trovo l'elemento in cui inserire il CAP
    cap_input = driver.find_element(By.CSS_SELECTOR, "#postcode")
    # Inserisco un CAP
    cap_input.send_keys("20124")
    time.sleep(3)
    # Trovo l'elemento del pulsante "Verifica CAP" utilizzando il selettore CSS
    verifica_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
    # Click pulsante "Verifica CAP"
    verifica_button.click()
    time.sleep(6)
    # Trovo l'elemento del link "Continua come ospite"
    continua_ospite_link = driver.find_element(By.CSS_SELECTOR, "a[data-remodal-action='close']")
    # Click sul link "Continua come ospite"
    continua_ospite_link.click()
    time.sleep(6)
    return None

# Codice raccolta prodotti Esselunga

In [5]:
# Apertura pagina chrome con webdriver
driver = webdriver.Chrome()
# URL di Esselunga
url_esselunga = 'https://www.esselungaacasa.it/ecommerce/nav/welcome/index.html'
# Indirizzamento alla pagina di Esselunga (è necessario fare il login con delle credenziali)
driver.get(url_esselunga)
time.sleep(3)
# Accesso alla pagina dei prodotti con inserimento di un CAP
access(driver)
# Trovo tutti i selettori delle varie categorie
categorie = driver.find_elements(By.CSS_SELECTOR, '.icon-right-open-big[ng-if="::menu2.hasChildren"]')
# Salvo la lunghezza per poter fare un ciclo
categorie_len = len(categorie)

In [6]:
# Creazione di dataframe vuoto
Esselunga = pd.DataFrame({'Product': [],
                          'Price': []})
# Ciclo for per iterare la raccolta prezzi su tutte le categorie
for elem in range(categorie_len):
    categorie[elem].click()
    time.sleep(6)
    scroll_down(driver)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    temp = find_all(soup)
    Esselunga = pd.concat([Esselunga, temp], ignore_index=True)
    driver.get(url_esselunga)
    time.sleep(6)
    access(driver)
    categorie = driver.find_elements(By.CSS_SELECTOR, '.icon-right-open-big[ng-if="::menu2.hasChildren"]')
    time.sleep(3)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[data-remodal-action='close']"}
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00858893+48451]
	(No symbol) [0x007EB8A1]
	(No symbol) [0x006F5058]
	(No symbol) [0x00720467]
	(No symbol) [0x0072069B]
	(No symbol) [0x0074DD92]
	(No symbol) [0x0073A304]
	(No symbol) [0x0074C482]
	(No symbol) [0x0073A0B6]
	(No symbol) [0x00717E08]
	(No symbol) [0x00718F2D]
	GetHandleVerifier [0x00AB8E3A+2540266]
	GetHandleVerifier [0x00AF8959+2801161]
	GetHandleVerifier [0x00AF295C+2776588]
	GetHandleVerifier [0x008E2280+612144]
	(No symbol) [0x007F4F6C]
	(No symbol) [0x007F11D8]
	(No symbol) [0x007F12BB]
	(No symbol) [0x007E4857]
	BaseThreadInitThunk [0x775900C9+25]
	RtlGetAppContainerNamedObjectPath [0x77917B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77917B1E+238]


In [27]:
# Chiusura pagina web
driver.close()

In [7]:
Esselunga

,Product,Price
0,"Esselunga Bio, cipolle dorate 500 g","Euro 3,16 / kg"
1,"Esselunga Bio, datterini biologici 300 g","Euro 8,27 / kg"
2,"Esselunga Bio, broccoletti biologici 500 g","Euro 6,36 / kg"
3,"Esselunga Bio, Insalatina 100 g","Euro 14,80 / kg"
4,"Esselunga Bio, 6 uova fresche biologiche","Euro 0,49 / pz"
...,...,...
13277,"Vileda, SuperMocio Fiocco 3Action","Euro 3,39 / pz"
13278,"WC Net, disincrostante gel 800 ml","Euro 2,24 / l"
13279,"Spontex, Panni Spugna 4 pezzi","Euro 0,40 / pz"
13280,"Cuki, Doppia Forza vaschette alluminio con cop...","Euro 0,51 / pz"


In [8]:
# Salvataggio dataframe in un file .cvs
Esselunga.to_csv("esselunga.csv")
Esselunga

,Product,Price
0,"Esselunga Bio, cipolle dorate 500 g","Euro 3,16 / kg"
1,"Esselunga Bio, datterini biologici 300 g","Euro 8,27 / kg"
2,"Esselunga Bio, broccoletti biologici 500 g","Euro 6,36 / kg"
3,"Esselunga Bio, Insalatina 100 g","Euro 14,80 / kg"
4,"Esselunga Bio, 6 uova fresche biologiche","Euro 0,49 / pz"
...,...,...
13277,"Vileda, SuperMocio Fiocco 3Action","Euro 3,39 / pz"
13278,"WC Net, disincrostante gel 800 ml","Euro 2,24 / l"
13279,"Spontex, Panni Spugna 4 pezzi","Euro 0,40 / pz"
13280,"Cuki, Doppia Forza vaschette alluminio con cop...","Euro 0,51 / pz"
